In [7]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from typing import List, Tuple, Any, Dict
from mcbo.optimizers.bo_builder import BoBuilder
from mcbo.tasks.task_base import TaskBase
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


df = pd.read_csv('train.csv')
print(df.shape)
df = df.dropna(subset=['Age', 'Fare', 'Sex'])
target_var = df.Survived
print(df.shape)

df = df[['Sex','Age', 'Fare']]
encode_cat_var = pd.get_dummies(df['Sex'])
df = df.drop(['Sex'], axis =1)
X = pd.concat([encode_cat_var, df], axis = 1)
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns) 

y = target_var
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Loss function
def binary_cross_entropy(y_true, y_pred):
    '''epsilon = 1e-15  # Small value to avoid numerical instability
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted probabilities to avoid taking the log of zero or one
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
'''
    cost = np.mean(np.abs(y_true - y_pred))  # Example: Mean absolute error
    return cost

# Feed-forward ANN
def forward_prop(W1, b1, W2, b2, X_data):
    n_features = X_data.shape[1]
    if W1.shape[1] != n_features:
        W1_shape_0 = W1.size / n_features 
        W1 = np.reshape(W1_shape_0 ,n_features)
    if W2.shape[1] != W1.shape[0]:
        W2_shape_0 = W2.size / W1.shape[0]
        W2 = np.reshape((W2_shape_0, W1.shape[0]))
    Z1 = np.dot(W1, X_data.T) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2).T 
    # Calculate the total number of weights
    total_weights = np.size(W1) + np.size(W2)

    # Calculate the total number of biases
    total_biases = np.size(b1) + np.size(b2)

    print("Total number of weights:", total_weights)
    print("Total number of biases:", total_biases)
    print(X_data.T.shape, 'shape data')

    
    return A2


# Evaluation function
def black_box_function(W1, b1, W2, b2):
    result = forward_prop(W1, b1, W2, b2, X_train)
    y_train_encoded = np.array([y_train]) # np.eye(2)[y_train]
    
    return binary_cross_entropy(y_train_encoded, result)

class CustomTask(TaskBase):
    @property
    def name(self) -> str:
        return 'Custom Task'
    
    def evaluate(self, x: np.ndarray) -> Tuple[pd.DataFrame ,np.ndarray]:
        y = np.zeros((len(x), 1))
        for ind in range(len(x)):
            x_ind = x.iloc[ind].to_numpy()  # Convert Series to NumPy array
            print("Evaluating with parameters:")
            print("x_ind:", x_ind)
            print('xind shape' , x_ind.shape)
            y[ind] = black_box_function(x_ind[:16].reshape((4, 4)), x_ind[16:20].reshape(4, 1), 
                                        x_ind[20:24].reshape(1, 4), x_ind[24:].reshape(1,1))
            
        return  x, y

                                             
    
    def get_search_space_params(self) -> List[Dict[str, Any]]:
        
        params = [{'name': f'W1_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(4) for j in range(4)]
        params.extend([{'name': f'b1_{i}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(4)])
        params.extend([{'name': f'W2_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(1) for j in range(4)])
        params.extend([{'name': f'b2_{i}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(1)])
        return params
        

    def get_parameter_names(self) -> List[str]:
        params = []
        for i in range(4):
            for j in range(4):
                params.append(f'W1_{i}{j}')
        params.extend([f'b1_{i}' for i in range(4)])
        for i in range(1):
            for j in range(4):
                params.append(f'W2_{i}{j}')
        params.extend([f'b2_{i}' for i in range(1)])
        return params

# Creating task and optimization
task = CustomTask()
searchspace = task.get_search_space()

epochs = 200
bo_builder = BoBuilder(model_id='gp_rd', acq_opt_id='is', acq_func_id='ei', tr_id='basic')
opt = bo_builder.build_bo(search_space=task.get_search_space(), n_init=200)


# Optimization loop
budget_eval = epochs
# Inside the optimization loop
weights_list = []
acc_list = []
for _ in range(budget_eval):
    x_next = opt.suggest()
    x_next_reshaped = np.array(x_next).reshape(1, -1)  # Reshape x_next to have shape (1, 25)
    x_next_df = pd.DataFrame(x_next_reshaped, columns=task.get_parameter_names())
    print(x_next_df.columns, 'columns88888888888888888888888888')
    y_next = task.evaluate(x_next_df)[1]
    weights_list.append(task.evaluate(x_next_df)[0])
    acc_list.append(y_next)
    print(y_next, '9999999999999999999999999999999999999999999999999999999999y next')
   
    opt.observe(x_next, y_next)
   

    
# Printing best result
print('Best parameters:', opt.best_x)
print('Best loss:', opt.best_y)

# Plotting version
fig, ax = plt.subplots(figsize=(7, 5))
y = opt.data_buffer.y.numpy()
regret_y = np.min(np.cumsum(y, axis=1), axis=0)
#plt.plot(np.arange(1,  budget_eval ), regret_y)
#plt.show()

(891, 12)
(714, 12)
Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [ 0.26816665  0.12477508  0.51374971 -0.01746884  0.89995347  0.80126042
 -0.04929814  0.01716701  0.46761137 -0.80253189 -0.30613378  0.50090338
 -0.23296444 -0.36458945 -0.59814475  0.19941071 -0.64721899 -0.07872478
  0.45871404  0.92853581  0.61031392  0.78733218  0.53295059 -0.17330597
  0.95619877]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [ 0.26816665  0.12477508  0.51374971 -0.01746884  0.89995347  0.80126042
 -0.04929814  0.01716701  0.46761137 -0.80253189 -0.30613378  0.50090338
 -0.23296444 -0.36458945 -0.59814475  0.19941071 -0.64721899 -

Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [ 0.42897801  0.68599278  0.39909576  0.06001799  0.28911381 -0.04842946
 -0.54878822  0.15028625 -0.3690942  -0.36698584 -0.62776534  0.60483736
  0.19765551 -0.80049793  0.19105809  0.09041832 -0.3885646   0.51707712
  0.10738588  0.04008644  0.58866225  0.56533046  0.25078861 -0.35587064
 -0.30493882]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [ 0.42897801  0.68599278  0.39909576  0.06001799  0.28911381 -0.04842946
 -0.54878822  0.15028625 -0.3690942  -0.36698584 -0.62776534  0.60483736
  0.19765551 -0.80049793  0.19105809  0.09041832 -0.3885646   0.51707712
  0.10738

Evaluating with parameters:
x_ind: [ 0.23676918  0.18190759  0.95488183  0.8550817   0.67834229  0.25218577
 -0.86234248  0.85238866  0.81343066 -0.4063312  -0.34540915  0.60443251
 -0.28530558  0.71161641  0.11828358 -0.64299744  0.38067703  0.31132994
 -0.55108158 -0.39401805  0.98491314  0.93949966  0.01030148 -0.82587262
  0.67115892]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
[[0.57317116]] 9999999999999999999999999999999999999999999999999999999999y next
Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [ 0.44235239 -0.80991387 -0.45507715  0.79769319  0.35596408 -0.26122147
  0.5524203  -0.85221017  0.33568699  0.62051248  0.79835642  0.78947343
  

Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [-0.97611478 -0.35276478 -0.59369991 -0.66701126 -0.63028801  0.36277119
 -0.52918726 -0.06595892 -0.907387    0.79564191 -0.25849906 -0.44738318
 -0.13518797 -0.27736668 -0.93845556 -0.12965228  0.44496168 -0.03219807
  0.60797458 -0.03030671 -0.24223842 -0.36987679  0.34179875 -0.57107987
 -0.32480218]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
[[0.49623849]] 9999999999999999999999999999999999999999999999999999999999y next
Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [-0.76963386 -0.46498832 -0.8799141  -0.32166934 -0.9270161  -0.2361781
 -

Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [-0.64776112  0.97976572  0.30887179 -0.02310337  0.32304411  0.00810137
 -0.7072654  -0.42689713 -0.29697347 -0.78871361  0.20541219 -0.10435011
 -0.98846453  0.90540332  0.61628923 -0.87529286 -0.64870139  0.30592972
  0.02024929 -0.10194978  0.58835362  0.50023759  0.38544648 -0.22731524
  0.1503176 ]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [-0.64776112  0.97976572  0.30887179 -0.02310337  0.32304411  0.00810137
 -0.7072654  -0.42689713 -0.29697347 -0.78871361  0.20541219 -0.10435011
 -0.98846453  0.90540332  0.61628923 -0.87529286 -0.64870139  0.30592972
  0.02024

...

Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [-0.93556175  0.05103887  0.95584248 -0.95780304 -0.24357952  0.54947636
  0.50066885 -0.17560267  0.84521187  0.60161995  0.78119741  0.53409273
  0.98011636  0.334536   -0.62797214  0.0153695  -0.68575814  0.69831429
 -0.04832798  0.74314208 -0.1452368   0.17177304 -0.94412263 -0.7667906
 -0.51938013]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [-0.93556175  0.05103887  0.95584248 -0.95780304 -0.24357952  0.54947636
  0.50066885 -0.17560267  0.84521187  0.60161995  0.78119741  0.53409273
  0.98011636  0.334536   -0.62797214  0.0153695  -0.68575814  0.69831429
 -0.048327

Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [ 0.8856198   0.12559621  0.04797742  0.90631427  0.57380503  0.89978279
 -0.55185262 -0.62570388 -0.7698404  -0.99103033 -0.43316525  0.2075329
 -0.10990572  0.79306793  0.53979595  0.53176842 -0.99125984 -0.58245907
 -0.78202053 -0.91193402  0.77495527 -0.1918909   0.855728   -0.63568417
  0.75409436]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [ 0.8856198   0.12559621  0.04797742  0.90631427  0.57380503  0.89978279
 -0.55185262 -0.62570388 -0.7698404  -0.99103033 -0.43316525  0.2075329
 -0.10990572  0.79306793  0.53979595  0.53176842 -0.99125984 -0.58245907
 -0.7820205

Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [-0.73015996 -0.07427313 -0.77248711  0.44055063 -0.86846327  0.95660278
 -0.69202702  0.89902002 -0.99247349 -0.69771983  0.95039074 -0.73196928
  0.57978669 -0.33579024  0.96044796  0.69289327  0.09731119 -0.82585842
 -0.04150375  0.5431878   0.53645645  0.91066435  0.95184932  0.52641373
  0.25628661]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [-0.73015996 -0.07427313 -0.77248711  0.44055063 -0.86846327  0.95660278
 -0.69202702  0.89902002 -0.99247349 -0.69771983  0.95039074 -0.73196928
  0.57978669 -0.33579024  0.96044796  0.69289327  0.09731119 -0.82585842
 -0.04150

Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [ 0.14453147  0.76435652 -0.44451467  0.59488209 -0.68297429 -0.55224061
 -0.57972693 -0.49755853  0.5580726  -0.62205246 -0.3491992  -0.27401437
  0.72450894  0.23027409 -0.77827331 -0.47275899 -0.43846164  0.79437115
  0.80633747 -0.99234334 -0.06621738  0.95204669  0.01972613 -0.14283506
 -0.37699204]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
[[0.48267428]] 9999999999999999999999999999999999999999999999999999999999y next
Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [ 0.84378357 -0.49899976  0.02389172 -0.00469272  0.6320631   0.88838859
 

[[0.49172945]] 9999999999999999999999999999999999999999999999999999999999y next
Index(['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12', 'W1_13',
       'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31', 'W1_32', 'W1_33',
       'b1_0', 'b1_1', 'b1_2', 'b1_3', 'W2_00', 'W2_01', 'W2_02', 'W2_03',
       'b2_0'],
      dtype='object') columns88888888888888888888888888
Evaluating with parameters:
x_ind: [ 0.39552878 -0.23579339 -0.36452614  0.5431345  -0.73454244  0.99583322
 -0.26801482 -0.82386608 -0.49126864  0.36582339 -0.61970851 -0.04106029
 -0.99620568 -0.99288673 -0.10684352  0.09074062 -0.9802781   0.4586184
 -0.03775338  0.06390142 -0.54701045 -0.1199862  -0.03132441  0.85360033
 -0.60004579]
xind shape (25,)
Total number of weights: 20
Total number of biases: 5
(4, 478) shape data
Evaluating with parameters:
x_ind: [ 0.39552878 -0.23579339 -0.36452614  0.5431345  -0.73454244  0.99583322
 -0.26801482 -0.82386608 -0.49126864  0.36582339 -0.61970851 -0.04106029
 -0

In [9]:
max(acc_list)

array([[0.57972482]])

In [10]:
x_next = pd.DataFrame(x_next_reshaped, columns=task.get_parameter_names())
print(len(x_next))

1


In [3]:
def weights_and_biases_matrix(df_pars):
    # Group the columns by their prefixes
    grouped_columns = df_pars.columns.to_series().groupby(lambda x: x.split('_')[0], sort=False)

    # Use dictionary comprehension to create the result dictionary
   # result_arrays = {prefix: np.vstack([df_pars[cols].columns, df_pars[cols].values])
    #                 for prefix, cols in grouped_columns.groups.items()}
    result_arrays = {prefix: np.array(df_pars[cols].values)
                     for prefix, cols in grouped_columns.groups.items()}

    return result_arrays

    #return weights_set_list#W1, weights_W2

weights_and_biases_results = (weights_and_biases_matrix(opt.best_x))
(weights_and_biases_results)


{'W1': array([[ 0.03040144,  0.76541645, -0.07756046, -0.6400424 ,  0.55365103,
          0.03805004,  0.54527582,  0.28889034,  0.92930014,  0.90562256,
          0.93051556,  0.08420812,  0.01683253,  0.15456392, -0.54305829,
         -0.78440319]]),
 'b1': array([[ 0.01150928, -0.38496867,  0.14046076,  0.7674008 ]]),
 'W2': array([[-0.04900145, -0.07966473, -0.94312315, -0.29997052]]),
 'b2': array([[-0.36293225]])}

In [4]:
def binarize_outcome(res_arr):
    return np.where(res_arr<.5,0,1)

def calculate_accuracy(actual_val, pred_val):
    correct = np.sum(actual_val == pred_val)
    total = len(actual_val)
    accuracy = correct / total
    
    return accuracy

'''preds_train_bo = forward_prop(**weights_and_biases_results, X_data=X_train)
binary_res = binarize_outcome(preds_train_bo)
print('accuracy =', calculate_accuracy(np.array(y_train), binary_res.flatten()))'''

"preds_train_bo = forward_prop(**weights_and_biases_results, X_data=X_train)\nbinary_res = binarize_outcome(preds_train_bo)\nprint('accuracy =', calculate_accuracy(np.array(y_train), binary_res.flatten()))"

In [5]:


# Feed-forward ANN
def forward_prop(W1, b1, W2, b2, X_data):
    n_features = X_data.shape[1]
   
    if W1.shape[1] != n_features:
        W1_shape_0 = int(W1.size / n_features) 
        W1 = np.reshape(W1, (W1_shape_0 ,n_features))
    if W2.shape[1] != W1.shape[0]:
        W2_shape_0 = int(W2.size / W1.shape[0])
        W2 = np.reshape(W2, (W2_shape_0, W1.shape[0]))
   
    Z1 = np.dot(W1, X_data.T) + b1.T
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2).T 
    return A2

for dataframe in weights_list:
    for_prop_res = forward_prop(*[dataframe[['W1_00', 'W1_01', 'W1_02', 'W1_03', 'W1_10', 'W1_11', 'W1_12',
                        'W1_13', 'W1_20', 'W1_21', 'W1_22', 'W1_23', 'W1_30', 'W1_31',
                        'W1_32', 'W1_33' ]].values,
                       dataframe[['b1_0', 'b1_1', 'b1_2', 'b1_3']].values,
                       dataframe[['W2_00', 'W2_01', 'W2_02', 'W2_03']].values,
                       dataframe[['b2_0']].values,
                       X_train])
    binary_result = binarize_outcome(for_prop_res)
    accuracy =calculate_accuracy(np.array(y_train), np.squeeze(binary_result))
    print(accuracy)
   


0.399581589958159
0.20711297071129708
0.600418410041841
0.7824267782426778
0.600418410041841
0.399581589958159
0.600418410041841
0.399581589958159
0.399581589958159
0.600418410041841
0.399581589958159
0.600418410041841
0.600418410041841
0.399581589958159
0.21338912133891214
0.40376569037656906
0.7510460251046025
0.399581589958159
0.399581589958159
0.399581589958159
0.600418410041841
0.399581589958159
0.399581589958159
0.600418410041841
0.399581589958159
0.399581589958159
0.600418410041841
0.7635983263598326
0.21338912133891214
0.600418410041841
0.399581589958159
0.24686192468619247
0.600418410041841
0.399581589958159
0.2217573221757322
0.7573221757322176
0.399581589958159
0.600418410041841
0.6799163179916318
0.24686192468619247
0.600418410041841
0.399581589958159
0.600418410041841
0.399581589958159
0.6569037656903766
0.600418410041841
0.399581589958159
0.600418410041841
0.41422594142259417
0.399581589958159
0.600418410041841
0.3891213389121339
0.48535564853556484
0.2405857740585774
0.4

In [6]:
print(forward_prop((*weights_list[7]), X_train))

TypeError: forward_prop() takes 5 positional arguments but 26 were given

In [ ]:
def weights_and_biases_matrix(df_pars):
    # Group the columns by their prefixes
    grouped_columns = df_pars.columns.to_series().groupby(lambda x: x.split('_')[0], sort=False)

    # Use dictionary comprehension to create the result dictionary
   # result_arrays = {prefix: np.vstack([df_pars[cols].columns, df_pars[cols].values])
    #                 for prefix, cols in grouped_columns.groups.items()}
    result_arrays = {prefix: np.array(df_pars[cols].values)
                     for prefix, cols in grouped_columns.groups.items()}

    return result_arrays

    #return weights_set_list#W1, weights_W2

weights_and_biases_results = (weights_and_biases_matrix(weights_list[7]))
(weights_and_biases_results)


In [ ]:
preds_train_bo = forward_prop(**weights_and_biases_results, X_data=X_train)
binary_res = binarize_outcome(preds_train_bo)
print('accuracy =', calculate_accuracy(np.array(y_train), binary_res.flatten()))

In [ ]:
from sklearn import metrics
print('accuracy =', calculate_accuracy(np.array(y_train), binary_res.flatten()))
confusion_matrix = metrics.confusion_matrix(y_train, binary_res.flatten())
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
def forward_prop(W1, b1, W2, b2, X_data):
    Z1 = np.dot(W1, X_data.T) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2).T 
    return A2


In [41]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from typing import List, Tuple, Any, Dict
from mcbo.optimizers.bo_builder import BoBuilder
from mcbo.tasks.task_base import TaskBase
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


from sklearn.datasets import make_classification

import numpy as np

def rosenbrock(x, y, z, w, a=1, b=100):
    return (a - x)**2 + b * (y - x**2)**2 + (a - z)**2 + b * (w - z**2)**2

def generate_synthetic_data(num_samples, num_features, num_minima):
    # Generate random samples within a predefined range
    X = np.random.uniform(-2, 2, size=(num_samples, num_features))
    
    # Calculate y values using the Rosenbrock function
    y = np.zeros(num_samples)
    for i in range(num_minima):
        x_min, y_min, z_min, w_min = np.random.uniform(-2, 2, size=4)  # Randomize the location of minima
        y += rosenbrock(X[:, 0] - x_min, X[:, 1] - y_min, X[:, 2] - z_min, X[:, 3] - w_min)
    
    # Determine a suitable threshold to balance the dataset
    threshold = np.percentile(y, 50)  # Set the threshold at the median of y
    
    # Threshold the y values to make them binary
    y = (y > threshold).astype(int)
    
    return X, y

# Example usage:
num_samples = 1000
num_features = 4
num_minima = 5

layer_sizes = np.array([num_features,5,1])  # Example list of layer sizes

X, y = generate_synthetic_data(num_samples, num_features, num_minima)
#print("X shape:", X.shape)
#print("y shape:", y.shape)
#print("Number of 0s:", np.sum(y == 0))
#print("Number of 1s:", np.sum(y == 1))

#y = target_var
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)




class flex_NN:
    def __init__(self, layers, learning_rate=0.1):
        self.layers = layers
        #self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(len(layers)-1)]
        #0self.biases = [np.random.randn(1, layers[i+1]) for i in range(len(layers)-1)]
        self.learning_rate = learning_rate


    def label_parameters(self):
        labels = {}
        for i in range(len(self.layers) - 1):
            labels[f"W{i}"] = f"{i}"
            labels[f"b{i}"] = f"{i}"
        return labels.keys()

    
    @staticmethod
    def parameters_dict(params_array):
        """
        Create a dictionary of parameters from an array containing weights and biases.

        Parameters:
        params_array -- array containing weights and biases in the order [W0, b0, W1, b1, ..., Wn, bn]

        Returns:
        parameters -- dictionary containing the parameters 'W0', 'b0', 'W1', 'b1', ..., 'Wn', 'bn'
        """
        num_layers = len(params_array) // 2
        parameters = {}
        for i in range(num_layers):
            parameters[f'W{i}'] = params_array[2*i]
            parameters[f'b{i}'] = params_array[2*i + 1]
        return parameters
    
    def blocks(self):
        blocks_info = []
        start_idx = 0
        for i in range(len(self.layers) - 1):
            # Weight block
            weight_block_size = self.layers[i] * self.layers[i+1]
            weight_block = (start_idx, start_idx + weight_block_size)
            reshape_weight = (self.layers[i+1], self.layers[i])

            # Bias block
            bias_block = (weight_block[1], weight_block[1] + self.layers[i+1])
            reshape_bias = (self.layers[i+1], 1)

            blocks_info.extend([weight_block, reshape_weight, bias_block, reshape_bias])

            start_idx = bias_block[1]  # Update start index for next iteration

        slicing, reshaping = blocks_info[::2], blocks_info[1::2] 
        #print(slicing, reshaping, 'slicing reshaping')
        return slicing, reshaping


    @staticmethod
    def forward_prop_sigmoid(X, parameters):
        """
        Perform forward propagation through each layer of a neural network.

        Parameters:
        parameters -- dictionary containing the parameters 'W0', 'b0', 'W1', 'b1', ..., 'Wn', 'bn'
        X -- input data (numpy array) of shape (input_size, m)

        Returns:
        A -- output of the last layer
        caches -- list of tuples containing the linear and activation values for each layer
        """
        caches = []
        A = X.T

        num_layers = len(parameters) // 2
        for i in range(num_layers):
            W = parameters[f'W{i}']
            b = parameters[f'b{i}']
            Z = np.dot(W, A) + b
            relu_Z, sigmoid_Z = np.maximum(0, Z), 1 / (1 + np.exp(-Z))
            A = relu_Z if i < num_layers - 1 else A = sigmoid_Z
            caches.append((Z, A))

        return A.T


    def softmax(Z):
        return np.exp(Z) / np.sum(np.exp(Z))

    # Loss function
    def binary_cross_entropy(y_true, y_pred):
        cost = np.mean(np.abs(y_true - y_pred))  # Example: Mean absolute error
        return cost


    def black_box_function(X_data, parameters):
        result = forward_prop(X_data, parameters)[0]
        y_train_encoded = np.array([y_train]) # np.eye(2)[y_train]
        return binary_cross_entropy(y_train_encoded, result)



class CustomTask(TaskBase):
    def __init__(self,layer_sizes): # pars
        self.layer_sizes = layer_sizes
        #self.pars = [np.array([]) for _ in range(len(layer_sizes) - 1)] #pars 
        

    @property
    def name(self) -> str:
        return 'Custom Task'
    
    def evaluate(self, x: np.ndarray) -> Tuple[pd.DataFrame ,np.ndarray]:
        slices, reshape_dims = blocks(self.layer_sizes)[0], blocks(self.layer_sizes)[1]
        y = np.zeros((len(x), 1))
        for ind in range(len(x)):
            x_ind = x.iloc[ind].to_numpy()  # Convert Series to NumPy array
            sliced_arrays = [x_ind[start:end] for start, end in slices]
            reshaped_arrays = tuple([sliced_array.reshape(shape) 
                               for sliced_array, shape in zip(sliced_arrays, reshape_dims)])
            feed_forward_res = 
            y[ind] = black_box_function(X_train, parameters_dict(reshaped_arrays))
        
        return x, y, parameters_dict(reshaped_arrays)
    
    def get_search_space_params(self) -> List[Dict[str, Any]]:
        params = []
        start_idx = 0
        for layer, size in enumerate(self.layer_sizes[:-1]):
            for i in range(size):
                for j in range(self.layer_sizes[layer + 1]):
                    params.append({'name': f'W{layer}_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1})
            for i in range(self.layer_sizes[layer + 1]):
                params.append({'name': f'b{layer}_{i}', 'type': 'num', 'lb': -1, 'ub': 1})
        return params
        

    def get_parameter_names(self) -> List[str]:
        params = []
        for layer, size in enumerate(self.layer_sizes[:-1]):
            for i in range(size):
                for j in range(self.layer_sizes[layer + 1]):
                    params.append(f'W{layer}_{i}{j}')
            for i in range(self.layer_sizes[layer + 1]):
                params.append(f'b{layer}_{i}')
        return params
  



# Creating task and optimization
task = CustomTask( layer_sizes) # label_parameters(layer_sizes)
searchspace = task.get_search_space()

nr_epochs = 50
bo_builder = BoBuilder(model_id='gp_rd', acq_opt_id='is', acq_func_id='ei', tr_id='basic')
opt = bo_builder.build_bo(search_space=task.get_search_space(), n_init=300)


# Optimization loop
budget_eval = nr_epochs
# Inside the optimization loop
weights_list = []
acc_list = []

def best_weights_and_biases_binary(my_func, epochs, X_data, y_data):
    best_accuracy = 0
    best_num_correct = 0
    best_weights_and_biases = None
    
    for _ in range(epochs):
        #print(_, 'epoch')
        x_next = opt.suggest()
        x_next_reshaped = np.array(x_next).reshape(1, -1)  # Reshape x_next to have shape (1, 25)
        x_next_df = pd.DataFrame(x_next_reshaped, columns=task.get_parameter_names())
        y_next = task.evaluate(x_next_df)[1]
        weights_and_biases = task.evaluate(x_next_df)[2]
        output_forward_prop = my_func(X_data, weights_and_biases)
        binary_output = np.where(output_forward_prop < .5, 0, 1).flatten()
        num_correct = np.sum(binary_output == y_data)
        accuracy = num_correct / y_data.size
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_num_correct = num_correct
            best_weights_and_biases = weights_and_biases
    
    return (best_num_correct, '/', y_data.size, '=', best_accuracy, 'accuracy', best_weights_and_biases)

        
print(best_weights_and_biases_binary(forward_prop, nr_epochs, X_train, y_train))
    
    
    



(452, '/', 670, '=', 0.6746268656716418, 'accuracy', {'W0': array([[ 0.61747465, -0.8674562 , -0.95422796, -0.81472931],
       [ 0.72869412, -0.03345382,  0.61470362,  0.55360359],
       [-0.7560461 ,  0.98158517,  0.42694676,  0.77023792],
       [-0.1054002 ,  0.03879079, -0.57591963, -0.74652309],
       [-0.63298496,  0.98329767,  0.15032252, -0.78395744]]), 'b0': array([[-0.39367465],
       [ 0.39656926],
       [-0.70400742],
       [-0.55185537],
       [ 0.37320287]]), 'W1': array([[-0.33073234, -0.9323665 , -0.2692264 ,  0.71539923,  0.61593644]]), 'b1': array([[0.88015392]])})


In [12]:
weights_list

[{'W0': array([[-0.449546  ,  0.34135783, -0.5619016 , -0.37716486],
         [-0.52861367, -0.76944023,  0.56170857,  0.57571429],
         [ 0.85004513,  0.91486834,  0.30438742,  0.13849693],
         [ 0.74335821, -0.07260929, -0.81785067, -0.28245186],
         [-0.79555513, -0.19285012,  0.16504906,  0.80845307]]),
  'b0': array([[-0.72707835],
         [-0.3997644 ],
         [ 0.804229  ],
         [-0.90695722],
         [ 0.09268959]]),
  'W1': array([[-0.31668882, -0.27789961,  0.98340558,  0.95843821, -0.00138986]]),
  'b1': array([[0.78374503]])},
 {'W0': array([[ 4.69042384e-01, -7.55223138e-01, -5.78396510e-01,
          -1.88710198e-01],
         [-3.77836134e-01, -5.61889169e-01,  5.54082093e-01,
           1.00734968e-01],
         [ 3.22591802e-05, -8.78177454e-01, -6.88934430e-01,
           7.81212907e-01],
         [-1.11015234e-01, -1.61300944e-01, -2.56222706e-01,
           5.73143561e-01],
         [ 6.60314896e-01, -6.00160730e-01,  6.07819222e-01,
          